In [16]:
from bs4 import BeautifulSoup
import requests
from newspaper import Article
from openpyxl import Workbook
import time

In [19]:
def china_list():
    ans = []
    query_url = "https://www.chinatimes.com/search/%E5%8F%B0%E7%81%A3%E7%8D%BC%E7%8C%B4?page="
    page = 1
    while True:
        soup = BeautifulSoup(requests.get(
            query_url + str(page)).text, "html.parser")
        arts = soup.select_one(".vertical-list")
        if arts is None:
            break
        arts = arts.select("li")
        if len(arts) == 0:
            break
        for art in arts:
            ans.append(art.select_one("a")["href"])
        page = page + 1
        if page == 19: #todo
            break
    return ans


In [22]:
wb = Workbook()
ws = wb.active
ws.title = "china"
ws_filte = wb.create_sheet("china_filte")


def temp_filter(s): return "遊憩干擾" in s or "衝突" in s or "遊憩活動" in s


for url in china_list():
    art = Article(url)
    art.download()
    art.parse()
    if ("遊客" not in art.text):
        continue
    output_arr = [
        art.publish_date.strftime(
            "%Y-%m-%d") if art.publish_date is not None else "None",
        art.title,
        art.text.replace("\n", " "),
        url
    ]
    ws.append(output_arr)
    if temp_filter(art.text):
        ws_filte.append(output_arr)
    time.sleep(1)


wb.save(filename="china.xlsx")


In [13]:
art = Article("https://www.chinatimes.com/realtimenews/20220210002667-260405?chdtv")
art.download()
art.parse()
art.publish_date.strftime("%Y-%m-%d")


'2022-02-10'